In [ ]:
import pandas as pd
import statistics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from datetime import datetime
from ast import literal_eval

getting data 

In [ ]:
#get topic modelling values
#change part of name '9t' for each number of topics
df_tm = pd.read_pickle("topic_modelling_values_9t.pkl")

#get review scores per listing from translated reviews
df_rt =  pd.read_pickle("agg_review_scores_translated.pkl")

#get listing data
data = pd.read_csv('listings.csv.gz', compression='gzip',)

EDA

In [ ]:
data.describe()

In [ ]:
# df1 with all numeric raw values
data.isnull().any(axis=1).sum()
#data.isna().sum()


In [ ]:
missing = data.isna().sum()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(missing)

In [ ]:
#count listings with bedrooms='NA' where description or name contains 'studio'
missing_bedrooms = data[data['bedrooms'].isnull()].index.tolist()
count =0
for i in missing_bedrooms:
    name = data['name'].iloc[i]
    desc = str(data['description'].iloc[i])
    if 'studio' in name:
        count += 1
    elif 'studio' in desc:
        count += 1
    else:
        continue
print(count)

In [ ]:
warnings.filterwarnings('ignore')
plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
sns.distplot(data['accommodates'])
plt.subplot(1,2,2)
sns.distplot(data['availability_90'])
plt.show()

feature transformation

In [ ]:
#host_since to amount of days they have been host
today = datetime.today()

def date_time(date):
    new_date = datetime.strptime(date, '%Y-%m-%d')
    days_tot = (today-new_date).days
    return days_tot

data['host_since'] = data.apply(lambda x: date_time(x.loc['host_since']), axis=1)

In [ ]:
#literal eval makes list variable from list in string format
def host_ver(x):
    try:
        leng = literal_eval(x)
    except:
        leng = []
    return leng
#get length of list (i.e. # of verifications host has)
data['host_verifications'] = data['host_verifications'].apply(host_ver)
data['host_verifications'] = data.apply(lambda x: len(x.loc['host_verifications']), axis=1)

In [ ]:
# listings where host has about text=1, listing without about text=0
data['host_about'] = data['host_about'].isna()
data['host_about'] = data['host_about'].map({True:0, False:1})



In [ ]:
#make price variable into float
data['price'] = data['price'].map(lambda price: price.replace('$', ''))
data['price'] = data['price'].map(lambda price: price.replace(',', ''))
data['price'] = pd.to_numeric(data['price'])

#remove rows with 0 value for price and get log price 
data.drop(data.index[data['price'] == 0], inplace = True)
data['log_price'] = np.log(data['price'])


In [ ]:
#bool for instant bookable
data['instant_bookable'] = data['instant_bookable'].map({'t': True, 'f': False})
#bool for identity verified
data['host_identity_verified'] = data['host_identity_verified'].map({'t': True, 'f': False})
#bool for superhost
data['host_is_superhost'] = data['host_is_superhost'].map({'t': True, 'f': False})

In [ ]:
#new dummy variable if bathroom is shared or not
data['bathrooms_text'] = data['bathrooms_text'].astype(str)
def bath_shared(text):
    if 'shared' in text:
        return 1
    else:
        return 0

data['bath_shared'] = data.apply(lambda x: bath_shared(x['bathrooms_text']), axis=1)

In [ ]:
#new dummy variable if bathroom is shared or not
def bath_number(text):
    if '1' in text:
        return 1
    elif '2' in text:
        return 2
    elif '3' in text:
        return 3
    elif '4' in text:
        return 4
    elif '5' in text:
        return 5
    elif 'half' in text:
        return 0.5
    else:
        return 0

data['bath_number'] = data.apply(lambda x: bath_number(x['bathrooms_text']), axis=1)

In [ ]:
#categories for accommodation type - new dataframe
def property_type(text):
    text = text.lower()
    if 'private' in text or 'room' in text:
        return 'type_private_room'
    elif 'shared' in text or 'hostel' in text:
        return 'type_shared_room'
    elif 'entire' in text:
        return 'type_entire_property'
    elif 'boat' in text:
        return 'type_boat'
    else:
        return 'type_other'

data['property_cat'] = data.apply(lambda x: property_type(x['property_type']), axis=1)
dpc = pd.get_dummies(data['property_cat'])


In [ ]:
#one-hot encoding for amenities

#get all unique values from amenities
import json
unique_val = set()
for i in data['amenities']:
    test = json.loads(i)
    for j in test:
        unique_val.add(j)

res = dict.fromkeys(unique_val, 0)

for i in data['amenities']:
    test = json.loads(i)
    for j in test:
        if j in unique_val:
            res[j] += 1
            
amenities_dict = dict(sorted(res.items(), key=lambda item: item[1], reverse=True))
sum(1 for i in amenities_dict.values() if i >= 1000)

# dummies for amenities that appear more than 2000 times
d = dict((k, v) for k, v in amenities_dict.items() if v >= 2000)
d=list(d.keys())

    
df_am = data[['id','amenities']].copy()
for i in d:
    def am_test(x):
        if i in x:
            return 1
        else:
            return 0
    col_name= 'amenities_'+ (i.replace(" ", "_"))
    df_am[col_name] = df_am['amenities'].apply(am_test)

#new dataframe with dummies 
df_am = df_am.drop(columns=['id', 'amenities'])


In [ ]:
#create dataframe with only relevant features
df1 = data[['id','accommodates', 'beds', 'bedrooms', 'bath_shared', 'bath_number', 'host_about', 'host_verifications', 'host_since', 'host_total_listings_count', 'host_identity_verified', 'host_is_superhost',  'instant_bookable','minimum_nights', 'maximum_nights', 'availability_365', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month', 'log_price']].copy()
#fill NA and drop remaining NA
df1['reviews_per_month'] = df1['reviews_per_month'].fillna(0)
df1['bedrooms'] = df1['bedrooms'].fillna(0)
df1 = df1.dropna()
df1.rename(columns = {'id':'listing_id'}, inplace = True)

In [ ]:
#dummies for neighbourhoods
dn = pd.get_dummies(data['neighbourhood_cleansed'])
#dataset with all dataframes with dummies attached
df1 = df1.join(dn)
df1 = df1.join(dpc)
df1 = df1.join(df_am)


In [ ]:
df1.to_pickle("dataset_final.pkl")